# SBTi-Finance Tool - Target Reporting
This notebook is an example of how to use the tool to calculate your portfolio's temperature scores and to save results to be submitted to SBTi's Target Validation Team (TVT) for validation and approval of your own Science Based Target.

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/07/Temperature-Scoring-Beta-Methodology.pdf), [guidance](https://sciencebasedtargets.org/financial-institutions) and the [technical documentation](http://getting-started.sbti-tool.org/) for more details on how to calculate portfolio temperature scores and how to submit targets to the SBTi for validation and approval. 

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/1_analysis_example.ipynb) or [Github](https://github.com/OFBDABV/SBTi/blob/master/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 

## Setting up
This section imports SBTi tools, import data providers, and load the portfolio.

In [1]:
!pip install SBTi

Looking in indexes: http://rdnexus1.ortec.finance:8081/nexus/repository/pypi-all/simple


In [2]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd

### Download resources for Google Colab environment


In [3]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/example_portfolio.csv", "data/example_portfolio.csv")
if not os.path.isfile("utils.py"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/utils.py", "utils.py")

try:  # Import statement when run in Google Colabs
    from utils import  print_aggregations, print_grouped_scores, anonymize, print_percentage_default_scores
except:  # Import statement when run locally
    from examples.utils import print_aggregations, print_grouped_scores, anonymize, print_percentage_default_scores

**Create a data provider**

By running the following cell, you load our example data. If you want to score your own portfolio, this is where you can upload data
from your data provider or your own data lake. Please refer to [Data Requirements](https://ofbdabv.github.io/SBTi/DataRequirements.html#) for a
full description of the data and formats needed.

By specifying a data provider, you can connect to the data source of your choice, e.g. CDP, Urgentem, your own data lake, etc. In this
example, the provider data is located in an Excel file.
Per default we use the example data downloaded in the previous step.
To replace the example data with your own data, first place your file in the data folder:
- When using Google Colab: click the folder logo in the left pane; hover your mouse of the 'data' folder; click the three dots; click upload; select your file.
- When running this notebook locally: place your file in the 'data' folder that is located in the same directory as this notebook.

Next, change the file location between the quotes below: replace 'data_provider_example' with your_filename (don't remove the double quotes in the cell below).

In [4]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")

**Load your portfolio**

In this example, the portfolio data is stored as a CSV file. If you wish to replace the portfolio file with your own portfolio file,
your CSV file should at least have a "company_id" column (the identifier of the company) and a "investment value" column (the amount
of money invested in each of the companies in your portfolio e.g. the value of the shares you hold). Please refer to [Data Requirements](https://ofbdabv.github.io/SBTi/DataRequirements.html#) for a full description of the data and formats needed.
To change the portfolio file, follow the same steps as for the data provider, described above.
Next, change the file location between the quotes below: replace '`example_portfolio`' with your_filename (don't remove the double quotes in the cell below).

In [5]:
portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")

The next cell displays the first 5 rows of data in your file.

In [6]:
portfolio.head(5)

,company_name,company_id,company_isin,weights,investment_value,engagement_target
0,Company AG,US0079031078,US0079031078,6.36%,35000000,True
1,Company AH,US00724F1012,US00724F1012,1.82%,10000000,NaN
2,Company AI,FR0000125338,FR0000125338,1.82%,10000000,NaN
3,Company AJ,US17275R1023,US17275R1023,1.82%,10000000,NaN
4,Company AK,CH0198251305,CH0198251305,1.82%,10000000,NaN


To load the data from the data provider, we need to convert the data of the portfolio.

In [7]:
companies = SBTi.utils.dataframe_to_portfolio(portfolio)


## Select calculation options
Configure computational settings for the temperature score. 
You can change the default score (fallback score in the code) and aggregation method below. The values specified here are the recommended defaults by the SBTi. If you want to overwrite the recommended settings see the comments for all the options per setting

In [8]:
# Settings                                              # Other Options: 
time_frames = [SBTi.interfaces.ETimeFrames.MID]         # reporting to TVT requires 'MID'
scopes = [EScope.S1S2, EScope.S1S2S3]                   # reporting to TVT requires 'S1S2' and 'S1S2S3'
fallback_score = 3.2                                    # The options of the default score are 3.2, 3.9 or 4.5
aggregation_method = PortfolioAggregationMethod.WATS    # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
grouping = ['sector']                                   # reporting to TVT requires 'sector'

In [9]:
temperature_score = TemperatureScore(
    time_frames=time_frames,
    scopes=scopes,
    fallback_score=fallback_score,
    aggregation_method=aggregation_method,
    grouping=grouping
)

## Calculate company and portfolio temperature scores

In [ ]:
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
amended_portfolio.head(5) # display top 5 rows

## Checks
In this example, the following scores are calculated:
- Portfolio temperature score
- Score for each sector
- Percentage covered by targets vs. default scores
- Portfolio coverage

In [ ]:
# Portfolio temperature score
print_aggregations(aggregated_scores)

In [ ]:
# Score per sector
print_grouped_scores(aggregated_scores)

In [ ]:
# Percentage of score based on default score
print_percentage_default_scores(aggregated_scores)

In [ ]:
# Portfolio coverage: Percentage of SBTs in portfolio
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
print("Portfolio coverage is {c:.2f}%".format(c=coverage))

## Save anonymized data for SBTi target validation
In order for the targets to be validated by SBTi, you can save your data locally. By running the anonymize function, you can replace company identifiers with meaningless substitutes.

In [ ]:
portfolio, provider = anonymize(portfolio, provider)

In order to store the portfolio and provider data locally, two options apply:
1. You are running the SBTi tool locally or from Google Colab
2. You are running the SBTi tool from a Docker container

If you run the SBTi tool locally or from Google Colab, you:
- Specify and filenames in the cell below
- Run the cell below

In [ ]:
portfolio_filename = 'portfolio.xlsx'
provider_filename = 'provider.xlsx'
portfolio.to_excel(portfolio_filename, index=False)

writer = pd.ExcelWriter(provider_filename, engine='openpyxl')
provider.data['fundamental_data'].to_excel(writer, sheet_name='fundamental_data')
provider.data['target_data'].to_excel(writer, sheet_name='target_data')
writer.save()

If you run the SBTi tool locally, you find the output files in the root folder of this notebook

If you run the SBTi tool from Google Colab, you:
- Click on the files icon in the left pane
- Click the three dots that appear after hovering over the file
- Download the file to your local machine